In [1]:
from appgeopy import *
from my_packages import *

In [55]:
prefix = "Choushui"
mainfolder = r"D:\1000_SCRIPTS\002_Manuscript_1"
validation_folder = os.path.join(mainfolder, "17_VERTICAL")
fig_savefolder = os.path.join(validation_folder, "Figures", "AverageVelocity_GPS_InSAR")
os.makedirs(fig_savefolder, exist_ok=True)
radius_of_buffer = 1000

o_displacement_fpath = os.path.join(validation_folder, f"CALIBRATED_o{prefix}_DISPLACEMENT_VERT_mm.pkl")
o_displacement_geodf = pd.read_pickle(o_displacement_fpath, compression="zip")

select_pointkey_fpath = glob(os.path.join(validation_folder, f"*{radius_of_buffer}*.pkl"))[0]
select_pointkey = pd.read_pickle(select_pointkey_fpath, compression="zip")

available_stations = select_pointkey["STATION"].unique()

In [57]:
merge_los_fpath = os.path.join(validation_folder, "VALIDATE_InSAR_GPS_VERT_Merge.xlsx")
gps_difference_fpath = os.path.join(mainfolder, "Differencing_GPS_LOS_mm_Manuscript001_v1.xlsx")

# ------------------------------------------------------------------------------
# Average Velocity Summary Cache
# ------------------------------------------------------------------------------
# Initialize a dictionary to store average velocity summary
avg_velo_cache = {
    "STATION": [],
    "GPS(mm/y)": [],
    "InSAR(mm/y)": []
}

# select_sheet = "TKJS"
for select_sheet in tqdm(available_stations):
    try:
        # ------------------------------------------------------------------------------
        df = pd.read_excel(merge_los_fpath, sheet_name=select_sheet, parse_dates=[0], index_col=[0])
        # ------------------------------------------------------------------------------
        gps_df = pd.read_excel(gps_difference_fpath, sheet_name=select_sheet, parse_dates=[0], index_col=[0])
        gps_df = gps_df.loc[:, ["U(mm)"]]
        # ------------------------------------------------------------------------------
        first_valid_idx = df["InSAR(mm)"].first_valid_index()
        # ------------------------------------------------------------------------------
        valid_df = df.loc[first_valid_idx:, :]
        valid_gps_df = gps_df.loc[first_valid_idx:, :]
        # ------------------------------------------------------------------------------
        # Interpolate to fill missing GPS data points
        filled_valid_df = valid_df.interpolate(method="piecewise_polynomial")
        filled_valid_gps_df = valid_gps_df.interpolate(method="piecewise_polynomial")
        # ------------------------------------------------------------------------------
        # Calculate cumulative displacement for the filled valid data
        cumdisp_valid_df = filled_valid_df.cumsum()
        cumdisp_valid_gps_df = filled_valid_gps_df.cumsum()
        # ------------------------------------------------------------------------------
        fulltime_cumdisp_df = dataframe_handle.convert_to_fulltime(cumdisp_valid_df)
        # ------------------------------------------------------------------------------
        insar_linear, insar_slope = analysis.get_linear_trend(fulltime_cumdisp_df["InSAR(mm)"])
        gps_linear, gps_slope = analysis.get_linear_trend(cumdisp_valid_gps_df["U(mm)"])
        
        # Add results to the average velocity cache
        for key, value in zip(list(avg_velo_cache.keys()), [select_sheet, gps_slope * 365.25, insar_slope * 365.25]):
            avg_velo_cache[key].append(value)
        
        # ------------------------------------------------------------------------------
        # Figure and Axes Setup
        # ------------------------------------------------------------------------------
        fig, ax = plt.subplots(1, 1, figsize=(11.7, 8.3*2/3))
        # ------------------------------------------------------------------------------
        # Plotting Data
        # ------------------------------------------------------------------------------
        ax.plot(cumdisp_valid_gps_df["U(mm)"], color="blue", marker=" ", label="GPS")
        ax.plot(gps_linear, color="deepskyblue")
        ax.plot(cumdisp_valid_df.loc[:, "InSAR(mm)"], color="orange", marker="o", label="InSAR")
        ax.plot(insar_linear, color="orangered")
        
        # Add text annotations for the slope values
        ax.text(0.025, 0.175, f"GPS: {gps_slope*365.25:.2f} (mm/y)", transform=ax.transAxes, ha="left", va="baseline", fontsize=12, color="deepskyblue")
        ax.text(0.025, 0.1, f"InSAR: {insar_slope*365.25:.2f} (mm/y)", transform=ax.transAxes, ha="left", va="baseline", fontsize=12, color="orangered")
        
        ax_min = np.min([cumdisp_valid_df["InSAR(mm)"].min(), cumdisp_valid_gps_df["U(mm)"].min()])
        ax_max = np.max([cumdisp_valid_df["InSAR(mm)"].max(), cumdisp_valid_gps_df["U(mm)"].max()])
        min_max_interval = abs(ax_max-ax_min)
        ax.set_ylim(bottom=ax_min-min_max_interval/4, top=ax_max+min_max_interval/4)
        ax.grid(axis="x", which="major", color="lightgrey")
        
        ylabel = "Cumulative\nDisplacement (mm)"
        visualize.configure_axis(ax, ylabel=ylabel, scaling_factor=0.7)
        visualize.configure_legend(ax, scaling_factor=0.7)
        
        # ------------------------------------------------------------------------------
        # Finalization
        # ------------------------------------------------------------------------------
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig.autofmt_xdate(rotation=90, ha="center")
        fig.suptitle(select_sheet, y=0.95, fontsize=20, fontweight="bold")
        
        savepath = os.path.join(fig_savefolder, select_sheet + ".png")
        visualize.save_figure(fig, savepath)
        
        plt.close()
    except Exception as e:
        print(select_sheet, e)
        pass

# ------------------------------------------------------------------------------
# Save Average Velocity Summary
# ------------------------------------------------------------------------------
# Create a DataFrame for the average velocity summary
avg_velo_summary = pd.DataFrame(avg_velo_cache)

summary_savename = f"VALIDATE_AverageVelocity_GPS_InSAR.xlsx"
summary_savepath = os.path.join(os.path.dirname(fig_savefolder), summary_savename)

# Save the average velocity summary to an Excel file
data_io.save_df_to_excel(df_to_save=avg_velo_summary, filepath=summary_savepath, sheet_name="AverageVelocity")

  0%|          | 0/39 [00:00<?, ?it/s]

ValueError: Worksheet named 'YWJS' not found